<a href="https://colab.research.google.com/github/J-Gann/donut_lora/blob/main/DocMeta_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update --yes
!apt-get install -y poppler-utils
%pip install -q -U transformers
%pip install pdf2image
%pip install -q gradio

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,314 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:14 http://archive.ubuntu.com/ubu

In [ ]:
import gradio as gr
from transformers import VisionEncoderDecoderModel, DonutProcessor
from pdf2image import convert_from_path
import torchvision as tv
import torch

task_start_token = "<s>"
eos_token = "</s>"
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
processor.tokenizer.add_special_tokens(
    {"additional_special_tokens": [task_start_token, eos_token]}
)

processor.feature_extractor.size = [2160, 3840]
model = VisionEncoderDecoderModel.from_pretrained("sodowo/doc_meta")

model.decoder.resize_token_embeddings(len(processor.tokenizer))
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = processor.tokenizer.convert_tokens_to_ids(
    ["<s>"]
)[0]

eval_model = model.eval()
eval_model.to("cuda")

In [24]:
def predict(file_path):
    image_data = convert_from_path(file_path, fmt="jpeg", first_page=1, last_page=1)[0]
    image_path = file_path.replace(".pdf", ".jpg")
    image_data.save(image_path)
    image_data = tv.io.read_image(image_path)
    image_data = image_data.permute(1, 2, 0)
    pixel_values = processor(image_data, return_tensors="pt").pixel_values
    pixel_values = pixel_values.squeeze()
    with torch.autocast(device_type='cuda', dtype=torch.float16):
        generated_ids = eval_model.generate(pixel_values.unsqueeze(0).to("cuda"))

    predicted_label = processor.batch_decode(
        generated_ids, skip_special_tokens=True
    )[0]

    return predicted_label, convert_from_path(file_path, fmt="jpeg", first_page=1, last_page=1)[0]

def process_pdf(file):
    if file is not None:
        return predict(file.name)
    return "No file uploaded."

with gr.Blocks() as demo:
    gr.Markdown("### Upload a PDF file for prediction")
    file_input = gr.File(label="Choose a PDF file", file_types=[".pdf"])
    output = gr.Textbox(label="Prediction Result")
    image_output = gr.Image(label="Processed Image")

    file_input.change(fn=process_pdf, inputs=file_input, outputs=[output, image_output])

demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3f8a106aec87278939.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
